<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/sft_peft_lora_gemma_2b_with_additional_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 训练 PEFT 模型时添加新标记到嵌入层和分词器


在这个示例中，我们将学习如何在向 tokenizer 和模型添加新标记时训练 LoRA 模型。
这是一个常见的用例，用于以下情况：
1. 使用新增标记进行指令微调，例如  `<|user|>`, `<|assistant|>`, `<|system|>`, `</s>`, `<s>`，以正确格式化对话。
2. 在特定语言上微调，在该语言的数据集上微调LLM时，添加语言特定的标记，例如添加韩语标记到词汇表中。
3. 在指令微调中返回特定格式的输出，以启用代理行为，例如新增标记 `<|FUNCTIONS|>`, `<|BROWSE|>`, `<|TEXT2IMAGE|>`, `<|ASR|>`, `<|TTS|>`, `<|GENERATECODE|>`, `<|RAG|>`。

在这种情况下，您需要将嵌入模块添加到 LORA 的 `target_modules` 中。PEFT 将负责保存嵌入层，其中包含新添加标记的权重，以及在特定初始化的嵌入层权重上训练的适配器权重。

In [1]:
!pip install -q peft transformers datasets accelerate wandb dataclass_csv flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 689.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 15

In [3]:
import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_PROJECT"] = "PeftExamples"
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import transformers
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    default_data_collator,
)
import torch
from dataclasses import dataclass, field
from typing import Optional
from dataclass_csv import DataclassReader
from torch.utils.data import Dataset, DataLoader

from enum import Enum

## Prepare Model and Tokenizer

现在，我们将添加 27 个新标记，并替换模型中现有的 pad、bos 和 eos 标记。






In [4]:
class SpecialTokens(str, Enum):
    begin_target = "<|begintarget|>"
    end_target = "<|endtarget|>"
    begin_context = "<|begincontext|>"
    end_context = "<|endcontext|>"
    system = "<|system|>"
    user = "<|user|>"
    begin_last_user_utterance = "<|beginlastuserutterance|>"
    end_last_user_utterance = "<|endlastuserutterance|>"
    begin_dsts = "<|begindsts|>"
    end_dsts = "<|enddsts|>"
    begin_dst = "<|begindst|>"
    end_dst = "<|enddst|>"
    begin_belief = "<|beginbelief|>"
    end_belief = "<|endbelief|>"
    begin_response = "<|beginresponse|>"
    end_response = "<|endresponse|>"
    begin_action = "<|beginaction|>"
    end_action = "<|endaction|>"
    begin_user_action = "<|beginuseraction|>"
    end_user_action = "<|enduseraction|>"
    sys_actions = "<|sysactions|>"
    begin_intent = "<|beginintent|>"
    end_intent = "<|endintent|>"
    begin_requested_slots = "<|beginrequestedslots|>"
    end_requested_slots = "<|endrequestedslots|>"
    pad_token = "<|pad|>"
    bos_token = "<|startoftext|>"

    @classmethod
    def list(cls):
        return [c.value for c in cls]

我们将微调 gemma-2B 模型。让我们加载分词器并添加特殊标记，然后加载基础模型并调整嵌入层大小以容纳新增标记。

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [6]:
!transformers-cli env

2024-04-04 06:12:53.000226: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 06:12:53.000315: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 06:12:53.002657: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-04 06:12:54.643327: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.

Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.38.2
- Platform: Linux-6.1.58+-x86_64-with-gl

In [ ]:
model_name = "google/gemma-2b"
#model_name = "google/gemman-2b-it"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    pad_token=SpecialTokens.pad_token.value,
    bos_token=SpecialTokens.bos_token.value,
    eos_token=SpecialTokens.end_target.value,
    additional_special_tokens=SpecialTokens.list(),
)

# just for from_pretrained model to inference
#max_memory = {0: "8GIB", "cpu": "30GB"}

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=False,
    #use_flash_attention_2=True, # leading to an error
    # FlashAttention only supports Ampere GPUs or newer
    #attn_implementation="flash_attention_2",
    #torch_dtype=torch.bfloat16,
    #device_map="auto",
    #max_memory=max_memory,
#)
).to(device)
print(model)
#print(model.hf_device_map)
print(model.config)
model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRM

In [7]:
print(model)
print(model.config)

#print(model.hf_device_map)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256027, 2048)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRMSNorm()
  )
  (

In [6]:
print(tokenizer)

GemmaTokenizerFast(name_or_path='google/gemma-2b', vocab_size=256000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endtarget|>', 'unk_token': '<unk>', 'pad_token': '<|pad|>', 'additional_special_tokens': ['<|begintarget|>', '<|endtarget|>', '<|begincontext|>', '<|endcontext|>', '<|system|>', '<|user|>', '<|beginlastuserutterance|>', '<|endlastuserutterance|>', '<|begindsts|>', '<|enddsts|>', '<|begindst|>', '<|enddst|>', '<|beginbelief|>', '<|endbelief|>', '<|beginresponse|>', '<|endresponse|>', '<|beginaction|>', '<|endaction|>', '<|beginuseraction|>', '<|enduseraction|>', '<|sysactions|>', '<|beginintent|>', '<|endintent|>', '<|beginrequestedslots|>', '<|endrequestedslots|>', '<|pad|>', '<|startoftext|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, sp

## Apply LoRA

**注**：IA3和LoRA差不多，增加了前馈层的参数权重微调，LoRA关注在embedding, self attention, lm_head层

In [8]:
config = LoraConfig(
    r=64, lora_alpha=128, lora_dropout=0.0, target_modules=["embed_tokens", "lm_head", "q_proj", "v_proj"]
)
#config = IA3Config(task_type=TaskType.SEQ_CLS, target_modules=["embed_tokens", "lm_head", "q_proj", "v_proj"], feedforward_modules=["down_proj"])

print(config)
model = get_peft_model(model, config)
print(model.print_trainable_parameters())
print(model)
print(model.config)


LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=64, target_modules={'q_proj', 'embed_tokens', 'v_proj', 'lm_head'}, lora_alpha=128, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)
trainable params: 40,406,400 || all params: 2,546,634,112 || trainable%: 1.5866590261082625
None
PeftModel(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): lora.Embedding(
          (base_layer): Embedding(256027, 2048)
          (lora_dropout): ModuleDict(
            (default): Identity()
          )
          (lora_A): ModuleDict()
          (lora_B): ModuleDict()
          (lora_embedding_

In [9]:
print(model.hf_device_map)

{'model.embed_tokens': 0, 'lm_head': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 0, 'model.layers.10': 0, 'model.layers.11': 0, 'model.layers.12': 0, 'model.layers.13': 'cpu', 'model.layers.14': 'cpu', 'model.layers.15': 'cpu', 'model.layers.16': 'cpu', 'model.layers.17': 'cpu', 'model.norm': 'cpu'}


`LoraConfig` 的配置，用于配置 LoRA（Low-Rank Adaptation）的相关选项。LoRA 是一种低秩适应技术，用于减少大型语言模型的参数量，并提高模型的效率和性能。以下是各个参数的含义解释：

1. `peft_type`: PEFT（Parameter-Efficient Fine-Tuning）类型。在这里，设置为 `PeftType.LORA`，表示使用 LoRA 进行微调。

2. `auto_mapping`: 自动映射。用于指定是否自动映射预训练模型的参数以适应新任务。

3. `base_model_name_or_path`: 基础模型名称或路径。用于指定要微调的基础语言模型的名称或路径。

4. `revision`: 模型修订版本。

5. `task_type`: 任务类型。在这里，未指定任务类型。

6. `inference_mode`: 推理模式。用于指定是否在推理时使用 LoRA。

7. `r`: LoRA 中的秩（rank）参数。用于控制低秩适应的参数量。

8. `target_modules`: 目标模块。指定了要应用低秩适应的模块名称集合。

9. `lora_alpha`: LoRA 中的 alpha 参数。用于控制低秩适应的强度。

10. `lora_dropout`: LoRA 中的 dropout 参数。用于控制低秩适应的正则化。

11. `fan_in_fan_out`: 是否使用扇入/扇出初始化。

12. `bias`: 偏置类型。指定了在低秩适应中使用的偏置类型。

13. `use_rslora`: 是否使用 RSLora。

14. `modules_to_save`: 要保存的模块列表。

15. `init_lora_weights`: 是否初始化 LoRA 权重。

16. `layers_to_transform`: 要转换的层列表。

17. `layers_pattern`: 层模式。

18. `rank_pattern`: 秩模式。

19. `alpha_pattern`: Alpha 模式。

20. `megatron_config`: Megatron 配置。

21. `megatron_core`: Megatron 核心。

22. `loftq_config`: LoFTQ 配置。

23. `use_dora`: 是否使用 DoRA。

24. `layer_replication`: 层复制。

这些参数用于配置 LoRA 过程中的各个方面，包括秩参数、模块选择、初始化方法等。通过调整这些参数，可以控制 LoRA 的行为，以满足特定任务和硬件环境的需求。

## Preapre Dataset

In [10]:
from datasets import load_dataset

dataset = load_dataset("smangrul/assistant_chatbot_dataset")
dataset = dataset["train"].train_test_split(0.2)

text_column = "context"
label_column = "target"
#max_length = 512
max_length = 256


def preprocess_function(examples):
    batch_size = len(examples[text_column])
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(examples[text_column])
    labels = tokenizer(text_target=targets, add_special_tokens=False)  # don't add bos token because we concatenate with inputs
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = model_inputs["input_ids"][i][:max_length]
        model_inputs["attention_mask"][i] = model_inputs["attention_mask"][i][:max_length]
        labels["input_ids"][i] = labels["input_ids"][i][:max_length]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]

Running tokenizer on dataset:   0%|          | 0/986 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/247 [00:00<?, ? examples/s]

In [11]:
print(dataset,train_dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['dialog_id', 'turn_id', 'context', 'target'],
        num_rows: 986
    })
    test: Dataset({
        features: ['dialog_id', 'turn_id', 'context', 'target'],
        num_rows: 247
    })
}) Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 986
})
{'dialog_id': '1_00024', 'turn_id': 1, 'context': '<|begincontext|><|beginlastuserutterance|>I will be going to eat with a friend what expensive restaurants do you recommend?<|endlastuserutterance|><|endcontext|>', 'target': '<|begintarget|><|begindsts|><|begindst|><|beginintent|>FindRestaurants<|endintent|><|beginbelief|>Restaurants^price_range->expensive<|endbelief|><|enddst|><|enddsts|><|beginuseraction|>INFORM->Restaurants^price_range~expensive|INFORM_INTENT->Restaurants^intent~FindRestaurants<|enduseraction|><|beginaction|>REQUEST->Restaurants^cuisine~Mexican~Chinese<|endaction|><|beginresponse|>What type of cuisine do you want? Mexican, Chinese or whic

In [12]:
# shuffle train datasets
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=8, pin_memory=True
)

In [13]:
next(iter(train_dataloader))

{'input_ids': tensor([[256002, 256002, 256002,  ..., 256017, 256001, 256001],
         [256002, 256002, 256002,  ..., 256017, 256001, 256001],
         [256002, 256002, 256002,  ..., 256017, 256001, 256001],
         ...,
         [256002, 256002, 256002,  ..., 256017, 256001, 256001],
         [256002, 256002, 256002,  ..., 256017, 256001, 256001],
         [256002, 256002, 256002,  ..., 256017, 256001, 256001]]),
 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         ...,
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1]]),
 'labels': tensor([[  -100,   -100,   -100,  ..., 256017, 256001, 256001],
         [  -100,   -100,   -100,  ..., 256017, 256001, 256001],
         [  -100,   -100,   -100,  ..., 256017, 256001, 256001],
         ...,
         [  -100,   -100,   -100,  ..., 256017, 256001, 256001],
         [  -100,   -100,   -100,  ..., 256017, 25600

In [14]:
tokenizer.decode(train_dataset[0]["input_ids"])

'<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad

# Train the model

In [ ]:
training_args = TrainingArguments(
    output_dir="sft_peft_lora_gemma_2b_with_added_tokens",
    num_train_epochs=2,
    save_total_limit=5,
    per_device_train_batch_size=8,
    warmup_steps=10,
    weight_decay=0.0001,
    dataloader_drop_last=True,
    #bf16=True,
    bf16=False,
    logging_steps=10,
    learning_rate=1e-5,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    remove_unused_columns=True,
    hub_model_id="weege007/sft_peft_lora_gemma_2b_with_added_tokens",
    push_to_hub=True,
    hub_private_repo=True,
)
print(training_args)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=default_data_collator,
)
# model.config.use_cache = False
trainer.train()

这是一个名为 `TrainingArguments` 的配置对象，用于配置训练过程的各种参数。以下是各个参数的含义解释：

1. `_n_gpu`: 使用的 GPU 数量。

2. `accelerator_config`: 加速器配置，用于分布式训练的配置项。

3. `adafactor`: 是否使用 Adafactor 优化器。

4. `adam_beta1`: Adam 优化器的 beta1 参数。

5. `adam_beta2`: Adam 优化器的 beta2 参数。

6. `adam_epsilon`: Adam 优化器的 epsilon 参数。

7. `auto_find_batch_size`: 是否自动查找最佳的批量大小。

8. `bf16`: 是否使用 BF16 数据类型。

9. `bf16_full_eval`: 是否在评估时使用完整的 BF16 计算。

10. `data_seed`: 数据种子。

11. `dataloader_drop_last`: DataLoader 是否丢弃最后一个不完整的批次。

12. `dataloader_num_workers`: DataLoader 的工作进程数量。

13. `dataloader_persistent_workers`: 是否使用持久化 DataLoader 工作进程。

14. `dataloader_pin_memory`: DataLoader 是否将数据加载到 CUDA 固定内存中。

15. `dataloader_prefetch_factor`: DataLoader 预取因子。

16. `ddp_backend`: 分布式数据并行（DDP）的后端。

17. `ddp_broadcast_buffers`: 是否广播缓冲区。

18. `ddp_bucket_cap_mb`: DDP 梯度累积的最大内存限制。

19. `ddp_find_unused_parameters`: DDP 是否查找未使用的参数。

20. `ddp_timeout`: DDP 超时时间。

21. `debug`: 调试选项。

22. `deepspeed`: DeepSpeed 配置。

23. `disable_tqdm`: 是否禁用 tqdm 进度条。

24. `dispatch_batches`: 分发批次。

25. `do_eval`: 是否执行评估。

26. `do_predict`: 是否执行预测。

27. `do_train`: 是否执行训练。

28. `eval_accumulation_steps`: 评估累积步数。

29. `eval_delay`: 评估延迟。

30. `eval_steps`: 评估步数。

31. `evaluation_strategy`: 评估策略。

32. `fp16`: 是否使用混合精度训练。

33. `fp16_backend`: 混合精度后端。

34. `fp16_full_eval`: 是否在评估时使用完整的混合精度计算。

35. `fp16_opt_level`: 混合精度优化级别。

36. `fsdp`: Fully Sharded Data Parallelism（FSDP）配置。

37. `fsdp_config`: FSDP 配置。

38. `fsdp_min_num_params`: FSDP 的最小参数数量。

39. `fsdp_transformer_layer_cls_to_wrap`: FSDP 要包装的 Transformer 层类。

40. `full_determinism`: 是否完全确定性。

41. `gradient_accumulation_steps`: 梯度累积步数。

42. `gradient_checkpointing`: 是否使用梯度检查点。

43. `gradient_checkpointing_kwargs`: 梯度检查点参数。

44. `greater_is_better`: 是否大值更好。

45. `group_by_length`: 是否按长度分组。

46. `half_precision_backend`: 半精度后端。

47. `hub_always_push`: 是否总是推送到 Hub。

48. `hub_model_id`: Hub 模型 ID。

49. `hub_private_repo`: 是否私有 Hub 仓库。

50. `hub_strategy`: Hub 策略。

51. `hub_token`: Hub 令牌。

52. `ignore_data_skip`: 是否忽略数据跳过。

53. `include_inputs_for_metrics`: 是否包含输入用于指标。

54. `include_num_input_tokens_seen`: 是否包含已见输入令牌数量。

55. `include_tokens_per_second`: 是否包含每秒令牌数。

56. `jit_mode_eval`: JIT 模式评估。

57. `label_names`: 标签名称。

58. `label_smoothing_factor`: 标签平滑因子。

59. `learning_rate`: 学习率。

60. `length_column_name`: 长度列名称。

61. `load_best_model_at_end`: 是否在结束时加载最佳模型。

62. `local_rank`: 本地排名。

63. `log_level`: 日志级别。

64. `log_level_replica`: 复制日志级别。

65. `log_on_each_node`: 每个节点记录日志。

66. `logging_dir`: 日志目录。

67. `logging_first_step`: 首次记录步骤。

68. `logging_nan_inf_filter`: 记录 NaN 和 Inf 过滤器。

69. `logging_steps`: 日志记录步数。

70. `logging_strategy`: 日志记录策略。

71. `lr_scheduler_kwargs`: 学习率调度器参数。

72. `lr_scheduler_type`: 学习率调度器类型。

73. `max_grad_norm`: 最大梯度范数。

74. `max_steps`: 最大步数。

75. `metric_for_best_model`: 最佳模型的指标。

76. `mp_parameters`: MP 参数。

77. `neftune_noise_alpha`: Neftune 噪声 alpha。

78. `no_cuda`: 是否不使用 CUDA。

79. `num_train_epochs`: 训练周期数。

80. `optim`: 优化器。

81. `optim_args`: 优化器参数。

82. `output_dir`: 输出目录。

83. `overwrite_output_dir`: 是否覆盖输出目录。

84. `past_index`: 过去索引。

85. `per_device_eval_batch_size`: 每个设备的评估批次大小。

86. `per_device_train_batch_size`: 每个设备的训练批次大小。

87. `prediction_loss_only`: 仅预测损失。

88. `push_to_hub`: 是否推送到 Hub。

89. `push_to_hub_model_id`: 推送到 Hub 的模型 ID。

90. `push_to_hub_organization`: 推送到 Hub 的组织。

91. `push_to_hub_token`: 推送到 Hub 的令牌。

92. `ray_scope`: Ray 作用域。

93. `remove_unused_columns`: 是否删除未使用的列。

94. `report_to`: 报告到的地方。

95. `resume_from_checkpoint`: 从检查点恢复。

96. `run_name`: 运行名称。

97. `save_on_each_node`: 是否在每个节点保存。

98. `save_only_model`: 是否仅保存模型。

99. `save_safetensors`: 是否保存安全张量。

100. `save_steps`: 保存步数。

101. `save_strategy`: 保存策略。

102. `save_total_limit`: 总保存限制。

103. `seed`: 随机种子。

104. `skip_memory_metrics`: 跳过内存指标。

105. `split_batches`: 分割批次。

106. `tf32`: 是否使用 TF32 数据类型。

107. `torch_compile`: 是否编译 Torch。

108. `torch_compile_backend`: Torch 编译后端。

109. `torch_compile_mode`: Torch 编译模式。

110. `torchdynamo`: TorchDynamo 配置。

111. `tpu_metrics_debug`: TPU 指标调试。

112. `tpu_num_cores`: TPU 核心数。

113. `use_cpu`: 是否使用 CPU。

114. `use_ipex`: 是否使用 IPEX。

115. `use_legacy_prediction_loop`: 是否使用传统的预测循环。

116. `use_mps_device`: 是否使用 MPS 设备。

117. `warmup_ratio`: 预热比率。

118. `warmup_steps`: 预热步数。

119. `weight_decay`: 权重衰减。

这些参数可以根据实际情况进行调整，以控制训练过程的各个方面，例如优化器、学习率、批次大小、训练周期数等。

# 在评估数据集的样本上检查模型输出

In [ ]:
import random

i = random.randint(0, len(dataset["test"]))
context = dataset["test"][i]["context"]

batch = tokenizer(context, return_tensors="pt")
batch = {k: v.to(device) for k, v in batch.items()}
model.eval()
output_tokens = model.generate(
    **batch,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
    top_p=0.95,
    top_k=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)
target_predicted = tokenizer.decode(output_tokens[0], skip_special_tokens=False).split("<|endcontext|>")[1]
target = dataset["test"][i]["target"]
print(f"{context=} \n\n {target_predicted=} \n\n {target=}")

context="<|begincontext|><|user|>Can you find me a place to eat please?<|system|>Where at? And what kind of cuisine are you craving?<|user|>Somewhere in SF, and I am really craving Thai food at the moment!<|system|>I found a bunch of restaurants, there's actually 10 that you might like in San Francisco, one of them being Baan Thai House & Wine Bar<|user|>How can I reach them? And what's their address?<|system|>You can reach them by phone at 415-379-4505 and visit them at 534 Irving Street<|beginlastuserutterance|>Great, that restaurant sounds good<|endlastuserutterance|><|endcontext|>" 

 target_predicted='<|begintarget|><|begindsts|><|begindst|><|beginintent|> FindRestaurants<|endintent|><|beginbelief|> Restaurants^city->SF~San Francisco|Restaurants^cuisine->Thai|Restaurants^restaurant_name->Baan Thai House & Wine Bar<|endbelief|><|enddst|><|enddsts|><|beginuseraction|> REQUEST->Restaurants^phone_number~|REQUEST->Restaurants^street_address~<|enduseraction|><|beginaction|> INFORM->Rest

# 保存适配器模型

当lora层应用于嵌入层时，相应的基础模型嵌入层也会被保存。

In [ ]:
trainer.push_to_hub()
trainer.model.push_to_hub(training_args.output_dir)

# 检查模型是否按预期加载并生成合理的输出

In [ ]:
from peft import PeftModel

inference_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    # use_flash_attention_2=True,
)
inference_model.resize_token_embeddings(len(tokenizer))

inference_model = PeftModel.from_pretrained(inference_model, "weege007/sft_peft_lora_gemma_2b_with_added_tokens")
inference_model.to(device)
inference_model.eval()

output_tokens = inference_model.generate(
    **batch,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
    top_p=0.95,
    top_k=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

target_predicted = tokenizer.decode(output_tokens[0], skip_special_tokens=False).split("<|endcontext|>")[1]
print(f"{context=} \n\n {target_predicted=} \n\n {target=}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

context="<|begincontext|><|user|>Can you find me a place to eat please?<|system|>Where at? And what kind of cuisine are you craving?<|user|>Somewhere in SF, and I am really craving Thai food at the moment!<|system|>I found a bunch of restaurants, there's actually 10 that you might like in San Francisco, one of them being Baan Thai House & Wine Bar<|user|>How can I reach them? And what's their address?<|system|>You can reach them by phone at 415-379-4505 and visit them at 534 Irving Street<|beginlastuserutterance|>Great, that restaurant sounds good<|endlastuserutterance|><|endcontext|>" 

 target_predicted='<|begintarget|><|begindsts|><|begindst|><|beginintent|> FindRestaurant<|endintent|><|beginbelief|> Restaurants^city->SF~San Francisco|Restaurants^cuisine->Thai|Restaurants^restaurant_name->Baan Thai House & Wine Bar<|endbelief|><|enddst|><|enddsts|><|beginuseraction|> REQUEST->Restaurants^phone_number~|REQUEST->Restaurants^street_address~<|enduseraction|><|beginaction|> INFORM->Resta

## 使用llama.cpp 测试下 gguf 支持

需要把新的 tokenizer.json tokenizer.model 格式化到 gguf中

## 使用 gemma.cpp 测试下

gemma.cpp 模型权重和分词器加载